In [4]:
import numpy as np
def get_inp(name):
    try:
        return np.loadtxt('inputs/day%s' % name)
    except:
        return open('inputs/day%s' % name).read()
import sys
sys.setrecursionlimit(100000)

# Day 1

In [16]:
def get_output(inp):
    if inp is None:
        return 0
    if len(inp)<=1 or inp is None:
        return 0
    if inp[1]==inp[0]:
        return (int(inp[0]) + get_output(inp[1:]))
    else:
        return get_output(inp[1:])

In [20]:
S=0
f = inp[0]
p = f
for c in inp[1:]:
    if c==p:
        S+=int(c)
    p=c
if c==f:
    S+=int(c)
print S

1119


## 1.1

In [ ]:
def get_output(inp, i):
    if inp is None:
        return 0
    if len(inp)<=1 or inp is None:
        return 0
    if inp[1]==inp[0]:
        return (int(inp[0]) + get_output(inp[1:]))
    else:
        return get_output(inp[1:])

In [3]:
import numpy as np
def get_output(inp, i):
    L = len(inp)
    if inp[i]==inp[np.mod(i+L/2,L)]:
        return int(inp[i])
    else:
        return 0

In [4]:
S=0
inp = '21752342814933766938172121674976879111362417653261522357855816893656462449168377359285244818489723869987861247912289729579296691684761143544956991583942215236568961875851755854977946147178746464675227699149925227227137557479769948569788884399379821111382536722699575759474473273939756348992714667963596189765734743169489599125771443348193383566159843593541134749392569865481578359825844394454173219857919349341442148282229689541561169341622222354651397342928678496478671339383923769856425795211323673389723181967933933832711545885653952861879231537976292517866354812943192728263269524735698423336673735158993853556148833861327959262254756647827739145283577793481526768156921138428318939361859721778556264519643435871835744859243167227889562738712953651128317624673985213525897522378259178625416722152155728615936587369515254936828668564857283226439881266871945998796488472249182538883354186573925183152663862683995449671663285775397453876262722567452435914777363522817594741946638986571793655889466419895996924122915777224499481496837343194149123735355268151941712871245863553836953349887831949788869852929147849489265325843934669999391846286319268686789372513976522282587526866148166337215961493536262851512218794139272361292811529888161198799297966893366553115353639298256788819385272471187213579185523521341651117947676785341146235441411441813242514813227821843819424619974979886871646621918865274574538951761567855845681272364646138584716333599843835167373525248547542442942583122624534494442516259616973235858469131159773167334953658673271599748942956981954699444528689628848694446818825465485122869742839711471129862632128635779658365756362863627135983617613332849756371986376967117549251566281992964573929655589313871976556784849231916513831538254812347116253949818633527185174221565279775766742262687713114114344843534958833372634182176866315441583887177759222598853735114191874277711434653854816841589229914164681364497429324463193669337827467661773833517841763711156376147664749175267212562321567728575765844893232718971471289841171642868948852136818661741238178676857381583155547755219837116125995361896562498721571413742'
for i in range(len(inp)):
    S+=get_output(inp,i)
print S

1420


# 2.1

In [30]:
inp = get_inp('2.1')

In [32]:
np.array(map(lambda x : x.max() - x.min(), inp)).sum()

45351.0

# 2.2

In [54]:
inp = get_inp('2.2')
def get_div(row):
    for i, e in enumerate(row):
        for e2 in np.hstack((row[:i],row[i+1:])):
            if e%e2==0:
                return e/e2
np.array(map(get_div, inp)).sum()

275.0

# 3.1

In [30]:
def get_path_to_1(inp):
    k = int(np.sqrt(inp))
    K = k/2 - (1-k%2)
    I = k%2
    p = inp-(k*k)
    return abs(K-min(p-1, k))+abs(K+I-max(p-k-1,0))

In [50]:
%time get_path_to_1(10)

CPU times: user 26 µs, sys: 20 µs, total: 46 µs
Wall time: 53.9 µs


3

# 3.2

In [2]:
#USELESS FINALLY
def get_coordinates(inp):
    if inp == 1:
        return 0,0
    k = int(np.sqrt(inp))
    K = k/2
    p = inp-(k*k)
    add = 1 if p==0 else 0 # if it's a square
    I = (-1)**(k%2+1)*(K-min(p-1,k)-add)
    J = (-1)**(k%2+1)*(K+k%2-max(p-k-1,0)-add)
    return I, J

In [3]:
def get_idx(I,J):
    if (I,J) == (0,0):
        return 1
    sign = np.sign(J-I) if J!=I else -1 #we want sign(0)=-1
    K = max(sign*J, -sign*I)-1
    k = K*2+1-min(0,sign)
    N = K +k**2+1
    N += -sign*I - 1*(min(sign, 0)) if -sign*I<=sign*J else -sign*J + k + 1
    return int(N)

In [4]:
M = {}
def compute_sum(I,J):
    if I==0 and J==0:
        return 1
    if '%d,%d' % (I,J) in M:
        return M['%d,%d' % (I,J)]
    this_idx = get_idx(I,J)
    S = 0
    for i in range(-1,2):
        for j in range(-1,2):
            if get_idx(I+i,J+j)<this_idx:
                if '%d,%d' % (I+i,J+j) not in M:
                    M['%d,%d' % (I+i,J+j)] = compute_sum(I+i,J+j)
                S+=M['%d,%d' % (I+i,J+j)]
    return S

def find_smallest_larger_than(X):
    k = 0
    while(True):
        if compute_sum(k,k)>X:
            sign=1
            if compute_sum(-k,-k)<X:
                sign = -1 
            value = compute_sum(sign*(k-1),sign*k)
            s = 2
            while X>value:
                value = compute_sum(sign*(k-min(s,2*k)),sign*(k-max(s-2*k,0)))
                s+=1
            return value
        k += 1

In [10]:
%time find_smallest_larger_than(10211726171726281982716820)

CPU times: user 26.3 ms, sys: 6.28 ms, total: 32.6 ms
Wall time: 32 ms


10327666307774372612997002L

# 4.1

In [12]:
def check_passphrase(paph):
    L = paph.split(' ')
    return len(L)==len(set(L))
inp = get_inp('4.1').split('\n')[:-1]
sum(map(check_passphrase,inp))

386

# 4.2

In [18]:
def doublecheck_passphrase(paph):
    L = map(lambda x : set(x), paph.split(' '))
    unique = reduce(lambda l, x: l.append(x) or l if x not in l else l, L, [])
    return len(unique)==len(L)
inp = get_inp('4.1').split('\n')[:-1]
sum(map(doublecheck_passphrase,inp))

208

# 5.1

In [47]:
inp = get_inp('5.1')
inp[inp[3]]
cursor = 0
L = len(inp)
step = 0
while True:
    prev = cursor
    cursor += inp[cursor]
    if inp[prev]>=3:
        inp[prev] -= 1
    else:
        inp[prev] += 1
    step +=1
    if cursor >= L or cursor <0:
        break
print step

28675390


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:2: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  from ipykernel import kernelapp as app
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:8: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:9: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:12: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:10: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


# 6.1

In [10]:
import numpy as np

import multiprocessing
from functools import partial

inp = get_inp('6.1').astype(int)
L=len(inp)
states =[] 
def choose_bank(inp):
    return inp.argmax()

def get_spread(idxn_b, iv):
    i, v = iv
    idx, n_b = idxn_b
    # print i, v, idx, n_b
    if i == idx:
        v = 0
    S=v+n_b/L
    if np.mod((i-idx)-1, L)<(n_b%L):
        S+=1
    return S

def spread_blocks(idx, inp):
    n_b = inp[idx] 
    #pool = multiprocessing.Pool()
    #inp = pool.map(partial(get_spread, (idx,n_b)), enumerate(inp))
    inp = map(partial(get_spread, (idx,n_b)), enumerate(inp))
    return np.array(inp) 

step = 0
while True:
    idx = choose_bank(inp) 
    inp = spread_blocks(idx, inp)
    hashinp = hash(inp.tostring())
    step += 1
    if hashinp in states:
        break
    states.append(hashinp)

# 6.1
#print step

# 6.2
print len(states)-states.index(hashinp)

8038


# 7.1

In [27]:
inp = get_inp('7.1').split('\n')[:-1]

In [30]:
def get_root(inp):
    inp = map(lambda x : x.split(' '), inp)
    leafs = dict([(l[0], {'w' : int(l[1].replace('(','').replace(')',''))}) for l in inp if '->' not in l])
    roots = dict([
        (l[0], {
                'w' : int(l[1].replace('(','').replace(')','')),
                'sons' : map(lambda x : x.replace(',',''), l[l.index('->')+1:])
               }) for l in inp if '->' in l])
    leafs_of_roots = [leaf for word in roots for leaf in roots[word]['sons']]
    for root in roots:
        if root not in leafs_of_roots:
            return root, roots, leafs

r, roots, leafs = get_root(inp)
print r

eugwuhl


# 7.2

In [31]:
def get_weights_of_sons(word):
    if word in leafs:
        return leafs[word]['w']
    else:
        return np.sum([get_weights_of_sons(w) for w in roots[word]['sons']]) + roots[word]['w']

In [53]:
def get_unbalanced_son(word):
    if word in leafs:
        return None, None, None
    L = [get_weights_of_sons(w) for w in roots[word]['sons']]
    value = len(set(L))==1
    if value:
        return None, None, None
    else:
        for i, l in enumerate(L):
            if L.count(l)==1:
                return roots[word]['sons'][i], l, L[np.mod((i+1),len(L))]

In [57]:
word = r
should_be = 0
v = 0
while word is not None:
    prev_word, diff = (word, v-should_be)
    word, should_be, v = get_unbalanced_son(word)
print "Bad value is %d, should be %d (weight of word %s)" % (roots[prev_word]['w'], roots[prev_word]['w'] + diff, prev_word)

Bad value is 428, should be 420 (weight of word drjmjug)


# 8.1

In [23]:
import operator
import numpy as np

ACTIONS = {'inc' : 1, 'dec' : -1}
CONDITIONS = {"==" : lambda x,y : x==y,
              "!=" : lambda x,y : x!=y,
              "<=" : lambda x,y : x<=y,
              ">=" : lambda x,y : x>=y,
              ">" : lambda x,y : x>y,
              "<" : lambda x,y : x<y
             }
variables = {'max_value' : -np.inf}
def code_parser(line):
    """
    <inp_variable> <action> <var_value> if <cond_variable> <condition> <cond_value>
    """
    inp_variable, action, var_value, _, cond_variable, condition, cond_value = line.split()
    variables[inp_variable] = 0 if inp_variable not in variables else variables[inp_variable]
    variables[cond_variable] = 0 if cond_variable not in variables else variables[cond_variable]
    if CONDITIONS[condition](variables[cond_variable], int(cond_value)):
        variables[inp_variable] = variables[inp_variable] + ACTIONS[action]*int(var_value)
        variables['max_value'] = max(variables['max_value'], variables[inp_variable])
        
inp = get_inp('8.1').split('\n')[:-1]
map(code_parser, inp)
max_value = variables['max_value']
del variables['max_value']
print "Answer to 8.1 : ", max(variables.itervalues())
print "Answer to 8.2 : ", max_value

Answer to 8.1 :  4567
Answer to 8.2 :  5636


# 9.1

In [37]:
inp = get_inp('9.1')[:-1]

In [72]:
import re
def remove_char(inp):
    return re.sub('!.{1}', '', inp)

def remove_garbage(inp):
    return re.sub('<.*?>', '', inp)

def count_in_garbage(inp):
    myre = re.compile(r'<.*?>')
    return np.sum(len(x) for x in map(lambda x : x[1:-1], myre.findall(inp)))

def get_subgroups(inp):
    return inp[1:-1].split(',')

def parenthetic_contents(string):
    """Generate parenthesized contents in string as pairs (level, contents)."""
    stack = []
    for i, c in enumerate(string):
        if c == '{':
            stack.append(i)
        elif c == '}' and stack:
            start = stack.pop()
            yield (len(stack), string[start + 1: i])

In [58]:
print "9.1 : ", np.sum([depth+1 for depth, val in parenthetic_contents(remove_garbage(remove_char(inp)))])np.sum([depth+1 for depth, val in parenthetic_contents(remove_garbage(remove_char(inp)))])

9.1 :  14204


In [73]:
print "9.2 : ", count_in_garbage(remove_char(inp))

9.2 :  6622
